In [1]:
from performance import Encoder
import pickle

%matplotlib inline
import matplotlib.pyplot as plt

from expression_encoding import interval_arithmetic

from timeit import default_timer as timer
import flags_constants as fc

# Convert `.nnet` to `.h5`

In [4]:
from NNet import NNet
from round_nnet import round_nnet_16bit

In [5]:
reference = NNet('./ExampleNNs/ACASXU_run2a_1_1_batch_2000.nnet')
test = NNet('./ExampleNNs/ACASXU_run2a_1_1_batch_2000.nnet')
test = round_nnet_16bit(test)

In [6]:
model_reference = reference.to_keras()
model_test = test.to_keras()

In [7]:
model_reference.save('./ExampleNNs/ACASXU_run2a_batch2000.h5')
model_test.save('./ExampleNNs/ACASXU_run2a_batch2000_float16.h5')

# Check Equivalence

In [8]:
fc.bounds_relu_early_stopping = True

In [9]:
input_his = [0.679858, 0.500000, 0.500000, 0.500000, -0.450000]
input_los = [0.600000, -0.500000, -0.500000, 0.450000, -0.500000]

In [10]:
reference = './ExampleNNs/ACASXU_run2a_batch2000.h5'
test = './ExampleNNs/ACASXU_run2a_batch2000_float16.h5'

In [11]:
enc = Encoder()
enc.encode_equiv(reference, test, input_los, input_his, 'optimize_diff_chebyshev')

In [12]:
enc.pretty_print()

### Vars ###
i_0_0: [0.6, 0.679858]
i_0_1: [-0.5, 0.5]
i_0_2: [-0.5, 0.5]
i_0_3: [0.45, 0.5]
i_0_4: [-0.5, -0.45]
A_x_0_0: [-999999, 999999]
A_x_0_1: [-999999, 999999]
A_x_0_2: [-999999, 999999]
A_x_0_3: [-999999, 999999]
A_x_0_4: [-999999, 999999]
A_x_0_5: [-999999, 999999]
A_x_0_6: [-999999, 999999]
A_x_0_7: [-999999, 999999]
A_x_0_8: [-999999, 999999]
A_x_0_9: [-999999, 999999]
A_x_0_10: [-999999, 999999]
A_x_0_11: [-999999, 999999]
A_x_0_12: [-999999, 999999]
A_x_0_13: [-999999, 999999]
A_x_0_14: [-999999, 999999]
A_x_0_15: [-999999, 999999]
A_x_0_16: [-999999, 999999]
A_x_0_17: [-999999, 999999]
A_x_0_18: [-999999, 999999]
A_x_0_19: [-999999, 999999]
A_x_0_20: [-999999, 999999]
A_x_0_21: [-999999, 999999]
A_x_0_22: [-999999, 999999]
A_x_0_23: [-999999, 999999]
A_x_0_24: [-999999, 999999]
A_x_0_25: [-999999, 999999]
A_x_0_26: [-999999, 999999]
A_x_0_27: [-999999, 999999]
A_x_0_28: [-999999, 999999]
A_x_0_29: [-999999, 999999]
A_x_0_30: [-999999, 999999]
A_x_0_31: [-999999, 999999]


## Bounds optimization

In [ ]:
teststart = timer()
enc.optimize_constraints(metric='chebyshev', use_lp_relaxation=True)
now = timer()
print("LP bounds: {}".format(now - teststart))

In [14]:
model_lp = enc.create_gurobi_model()

In [15]:
teststart = timer()
enc.optimize_constraints(metric='chebyshev', use_lp_relaxation=False)
now = timer()
print("MILP bounds: {}".format(now - teststart))

A_x_1_0: [-2.266290730480617, -0.2921558832888165]
A_x_1_1: [-0.40121584683850087, -0.03137147046027486]
A_x_1_2: [-2.8975183022286766, 0.6187956560390852]
A_x_1_3: [-5.53636039927164, 1.3802037938846874]
A_x_1_4: [-2.619666540926955, -0.11180387278897418]
A_x_1_5: [-5.239536666601546, -1.1720033731804813]
A_x_1_6: [-0.21318041945613997, 0.04008998734498512]
A_x_1_7: [-11.898521952552402, 4.9896324628229385]
A_x_1_8: [-4.499338573757363, -2.0748515380283887]
A_x_1_9: [0.0017684874679821938, 3.4063721393776385]
A_x_1_10: [-0.2712627448919718, -0.011421623422489265]
A_x_1_11: [-1.7377134556938896, 0.07806888231075743]
A_x_1_12: [-9.791312012323509, -0.9672181941598262]
A_x_1_13: [-9.92803241596992, -2.3418570241240144]
A_x_1_14: [-3.815273915003602, -1.3765446920559974]
A_x_1_15: [-4.7785154794065186, 1.7867567977964374]
A_x_1_16: [-1.5744092128482852, 1.3660447119590695]
A_x_1_17: [-1.3587104489391841, 0.340253359417249]
A_x_1_18: [-3.59204953770113, -0.4163027402692604]
A_x_1_19: [-4.7

In [16]:
teststart = timer()
model_mip = enc.create_gurobi_model()
now = timer()
print('Model building: {}'.format(now - teststart))

Model building: 2.394110499999897


In [19]:
eps = 0.05
# stop, when proven that objective must be <= eps
model_mip.setParam('BestBdStop', eps)
# stop, as soon as feasible solution with objective > eps is found
model_mip.setParam('BestObjStop', eps + 1e-4)

Parameter BestBdStop unchanged
   Value: 0.05  Min: -inf  Max: inf  Default: inf
Changed value of parameter BestObjStop to 0.050100000000000006
   Prev: 0.05  Min: -inf  Max: inf  Default: -inf


In [18]:
model_mip.setParam('TimeLimit', 300)
model_mip.optimize()

Changed value of parameter TimeLimit to 300.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 3613 rows, 1847 columns and 32539 nonzeros
Model fingerprint: 0xe56ee4ce
Variable types: 1232 continuous, 615 integer (615 binary)
Coefficient statistics:
  Matrix range     [1e-05, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [2e-03, 1e+03]
  RHS range        [5e-03, 1e+03]
Presolve removed 1014 rows and 338 columns
Presolve time: 0.38s
Presolved: 2599 rows, 1509 columns, 25688 nonzeros
Variable types: 1031 continuous, 478 integer (478 binary)

Root relaxation: objective 4.677261e+01, 830 iterations, 0.10 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   46.77261    0   68          -   46.77261      -     